In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')

# Preface

## What is a NEET?

A NEET, [defined as](https://www.oecd.org/en/data/indicators/youth-not-in-employment-education-or-training-neet.html) an individual (typically young) that is *not in education, employment or training*. While empirical definitions for NEETs vary a bit--depending on age ranges to include (16 to 24? 20 to 29?), or how to define the activity criteria (unemployed? or not in labor force?)--the general NEET rates shouldn't vary too considerably. Rather, the directional trends should stay the same. 

Nonetheless, we'll try out multiple definitions throughout this project. As a reference, the OECD [publishes NEET rates](https://www.oecd.org/en/data/indicators/youth-not-in-employment-education-or-training-neet.html?oecdcontrol-dec63071aa-var6=15_29) at the country-level. Let's briefly take a look at the overall NEET rates by country in 2022, looking at 15-29 year-olds.

In [8]:
oecd_overall = pd.read_csv('../datasets/OECD_15to29_NEET.csv', skiprows=3, names = ['country', 'rate'])
oecd_overall.head()

,country,rate
0,Türkiye,27.931236
1,Italy,22.926674
2,Mexico,19.481358
3,Spain,16.476675
4,Greece,16.008015
